In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import cv2
import tensorflow as tf
from tensorflow import keras
from keras import layers
from functools import partial

import os
import xml.etree.ElementTree as ET

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, ReLU, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.applications import Xception
from tensorflow.keras.utils import to_categorical

import warnings
warnings.filterwarnings('ignore')
AUTO = tf.data.experimental.AUTOTUNE

In [2]:
# # Extract zip file downloaded from kaggle
# from zipfile import ZipFile
# data_path = r'C:\Users\haris\AI\ML\Projects\Dog Breed Classification\archive.zip'

# with ZipFile(data_path, 'r') as zip:
# 	zip.extractall()
# 	print('The data set has been extracted.')

In [52]:
# Path to the folders containing images and annotations
annotations_folder = r'C:\Users\haris\AI\ML\Projects\Data\Dog Breed Classification\annotations'
images_folder = r'C:\Users\haris\AI\ML\Projects\Data\Dog Breed Classification\images'

# Initialize lists to store image data and labels
images = []
labels = []

# Parse XML annotations and load images
for breed_folder in os.listdir(annotations_folder):
    breed_path = os.path.join(annotations_folder, breed_folder)
    for annotation_file in os.listdir(breed_path):
        annotation_path = os.path.join(breed_path, annotation_file)
        
        tree = ET.parse(annotation_path)
        root = tree.getroot()
        
        breed = root.find('object/name').text
        # xmin = int(root.find('object/bndbox/xmin').text)
        # ymin = int(root.find('object/bndbox/ymin').text)
        # xmax = int(root.find('object/bndbox/xmax').text)
        # ymax = int(root.find('object/bndbox/ymax').text)
        
        img_path = os.path.join(images_folder, breed_folder, annotation_file + '.jpg')
        image = cv2.imread(img_path)
        image = cv2.resize(image, (224, 224))  # Resize to desired size
        image = image/255
        
        images.append(image)
        labels.append(breed)

In [55]:
no_of_labels = (len(labels))
print(no_of_labels)

20580


In [68]:
# Convert labels to numerical format
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
num_classes = len(label_encoder.classes_)
labels_encoded=labels_encoded.reshape(no_of_labels, 1)
print(labels_encoded.shape)
print(labels_encoded[0])

(20580, 1)
[17]


In [65]:
# Preprocess the labels
labels_encoded_onehot = to_categorical(labels_encoded, num_classes)
print(labels_encoded_onehot.shape)

(20580, 120)


In [67]:
# Print the first row
print("First row of labels_encoded_onehot:")
print(labels_encoded_onehot[0])

# Print the shape of the array
print("Shape of labels_encoded_onehot:", labels_encoded_onehot.shape)

First row of labels_encoded_onehot:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Shape of labels_encoded_onehot: (20580, 120)


In [46]:
# Convert the images list to a NumPy array
images = np.array(images)


In [47]:
print(labels_encoded)
print((labels_encoded.shape))
print((images.shape))

[[0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]]
(10, 1)
(10, 224, 224, 3)


In [48]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)

In [50]:
# Split data into training, validation, and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(images, labels_encoded, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Preprocess the labels
y_train_onehot = to_categorical(y_train, num_classes)
y_val_onehot = to_categorical(y_val, num_classes)
y_test_onehot = to_categorical(y_test, num_classes)

print((y_train_onehot))
print((X_train).shape)



[[1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]]
(7, 224, 224, 3)


In [41]:
print(y_temp.shape)

(2, 1)


In [18]:
# Build Simple CNN
def build_simple_cnn(input_shape, num_classes):
    model = Sequential()
    
    # Convolutional blocks
    for _ in range(4):
        model.add(Conv2D(64, (3, 3), padding='same', input_shape=input_shape))
        model.add(BatchNormalization())
        model.add(ReLU())
        model.add(MaxPooling2D((2, 2)))
        model.add(Dropout(0.25))
    
    # Flatten and fully connected layers
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    
    return model

# Build Xception model with transfer learning
def build_xception(input_shape, num_classes):
    base_model = Xception(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in base_model.layers:
        layer.trainable = False
    model = Sequential()
    model.add(base_model)
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))
    return model

# Specify input shape and number of classes
input_shape = (224, 224, 3)  # Example input shape
num_classes = 120  # Number of dog breeds

# Build and compile Simple CNN model
simple_cnn_model = build_simple_cnn(input_shape, num_classes)
simple_cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Build and compile Xception model
xception_model = build_xception(input_shape, num_classes)
xception_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# # Data augmentation for Simple CNN
# datagen = tf.keras.preprocessing.image.ImageDataGenerator(
#     horizontal_flip=True,
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     fill_mode='nearest',
#     preprocessing_function=tf.keras.applications.xception.preprocess_input
# )

# # Create data generators using tf.data
# def create_data_generator(images, labels, batch_size, is_training=True):
#     dataset = tf.data.Dataset.from_tensor_slices((images, labels))
#     if is_training:
#         dataset = dataset.shuffle(buffer_size=len(images)).repeat()

#     dataset = dataset.map(lambda x, y: (datagen.random_transform(x), y), num_parallel_calls=tf.data.AUTOTUNE)
#     dataset = dataset.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
#     return dataset

# # Create data generators
# train_data_generator = create_data_generator(X_train, y_train_onehot, batch_size=32)
# val_data_generator = create_data_generator(X_val, y_val_onehot, batch_size=32, is_training=False)
# test_data_generator = create_data_generator(X_test, y_test_onehot, batch_size=32, is_training=False)

# Train Simple CNN
history_simple_cnn = simple_cnn_model.fit(X_train, y_train_onehot, epochs=10, validation_data=(X_val, y_val_onehot))

# Train Xception
history_xception = xception_model.fit(X_train, y_train_onehot, epochs=10, validation_data=(X_val, y_val_onehot))

# Evaluate models
simple_cnn_loss, simple_cnn_acc = simple_cnn_model.evaluate(X_test, y_test_onehot, verbose=2)
xception_loss, xception_acc = xception_model.evaluate(X_test, y_test_onehot, verbose=2)

print("Simple CNN - Test accuracy:", simple_cnn_acc)
print("Xception - Test accuracy:", xception_acc)

MemoryError: Unable to allocate 8.08 GiB for an array with shape (14406, 224, 224, 3) and data type float32